# Run Open Source Inference 

This notebook runs inference on the open source models using the multi-gpu process implemented in the modelinference file. This file requires the llm-base environment to run correctly. The results of the backtest are stored in the Data folder under each model folder. 

We test with a handful of open source models:

- Llama 3.2 3B Instruct
- Qwen 2.5 7B Instruct
- Deepseek
- 

In [1]:
import model_inference
import prompts
import importlib
import datetime
import torch
from huggingface_hub import login
import utils.model_helper as mh

from transformers import BitsAndBytesConfig
from accelerate import Accelerator, notebook_launcher

In [2]:
importlib.reload(model_inference)
importlib.reload(prompts)
importlib.reload(mh)

<module 'utils.model_helper' from '/project/utils/model_helper.py'>

### Set up the environment
Log into Huggingface and check the number of GPUs available

In [3]:
# Log into Huggingface
with open('pass.txt') as p:
    hf_login = p.read()
    
hf_login = hf_login[hf_login.find('=')+1:hf_login.find('\n')]
login(hf_login, add_to_git_credential=False)

In [4]:
print(f'Torch version: {torch.__version__}')
print(f'Device Count: {torch.cuda.device_count()}')

Torch version: 2.1.2.post300
Device Count: 4


### Run 1: Llama 3.2 Earning analysis - Base


In [5]:
# Create the run config
run_config = {
    'model_hf_id': 'meta-llama/Llama-3.2-3B-Instruct',
    'model_s3_loc': 'llama',
    'model_reload': False,
    'model_quant': None,
    'system_prompt': prompts.SYSTEM_PROMPTS['BASE_EARN'],
    'multi-gpu':True,
    'dataset': 'data_quarterly_pit_indu_blended',
    'data_location': 'data_quarterly_pit_indu_refresh_blended.json'
}

In [6]:
run_name = f"{run_config['model_s3_loc']}_{run_config['dataset']}"
ir = model_inference.InferenceRun(run_name, run_config)

# Create the prompts and save to the Data folder
prompt_set = ir.create_all_prompts(force_refresh=True, is_save_prompts=True)


Requesting all datasets...
Saving data...


In [7]:
# Run the multi-gpu model with notebook_launcher
notebook_launcher(ir.run_multi_gpu, num_processes=torch.cuda.device_count())

Launching training on 4 GPUs.
llamallama

llama
llama


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Waiting...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/887 [00:00<?, ?it/s]

Memory footprint: 6.4 GB
Waiting...
starting backtest...
starting backtest...
starting backtest...
starting backtest...


888it [26:41,  2.21s/it]                         

Finished run...
Finished run...
Finished run...
Finished run...
Gathered results...Gathered results...Gathered results...Gathered results...



Finished run in 0:28:23.145980
Called Save run
called log
Saved bclarke16/tmp/fs/logs/Results_2025-03-30 10:05:34.896587.json
Run Completed!


888it [28:23,  1.92s/it]
[2025-03-30 10:34:55,568] torch.distributed.elastic.multiprocessing.api: [WARNING] Closing process 2401 via signal SIGTERM
[2025-03-30 10:34:55,569] torch.distributed.elastic.multiprocessing.api: [WARNING] Closing process 2402 via signal SIGTERM
[2025-03-30 10:34:55,570] torch.distributed.elastic.multiprocessing.api: [WARNING] Closing process 2403 via signal SIGTERM


### Run 2: Llama 3.2 Earning analysis - Chain of Thought


In [5]:
run_config = {
    'model_hf_id': 'meta-llama/Llama-3.2-3B-Instruct',
    'model_s3_loc': 'llama',
    'model_reload': False,
    'model_quant': None,
    'system_prompt': prompts.SYSTEM_PROMPTS['COT_EARN'],
    'multi-gpu':True,
    'dataset': 'data_quarterly_pit_indu_blended_cot',
    'data_location': 'data_quarterly_pit_indu_refresh_blended.json'
}

In [6]:
run_name = f"{run_config['model_s3_loc']}_{run_config['dataset']}"
ir = model_inference.InferenceRun(run_name, run_config)

# Create the prompts and save to the Data folder
prompt_set = ir.create_all_prompts(force_refresh=True, is_save_prompts=True)

Requesting all datasets...
Saving data...


In [7]:
# Run the multi-gpu model with notebook_launcher
notebook_launcher(ir.run_multi_gpu, num_processes=torch.cuda.device_count())

Launching training on 4 GPUs.
llama
llama
llama
llama


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Waiting...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

 98%|█████████▊| 868/887 [58:12<01:21,  4.30s/it]

Finished run...


888it [59:41,  4.45s/it]                         

Finished run...
Gathered results...Gathered results...Gathered results...Gathered results...



Finished run in 0:59:41.735791
Called Save run
called log
Saved bclarke16/tmp/fs/logs/Results_2025-03-30 10:36:38.332729.json
Run Completed!


888it [59:42,  4.03s/it]
[2025-03-30 11:37:17,363] torch.distributed.elastic.multiprocessing.api: [WARNING] Closing process 3624 via signal SIGTERM
[2025-03-30 11:37:17,365] torch.distributed.elastic.multiprocessing.api: [WARNING] Closing process 3626 via signal SIGTERM
[2025-03-30 11:37:17,365] torch.distributed.elastic.multiprocessing.api: [WARNING] Closing process 3627 via signal SIGTERM


### Run 3: DeepSeek R1 Qwen 7B Base

In [15]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"

)

model_helper = mh.ModelHelper('tmp/fs')
model_helper.get_model_and_save('deepseek-ai/DeepSeek-R1-Distill-Qwen-7B',
                                'deepseek7B', 
                                'Data',
                                True,
                                quant_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

None


In [5]:
run_config = {
    'model_hf_id': 'deepseek-ai/DeepSeek-R1-Distill-Qwen-7B',
    'model_s3_loc': 'deepseek7B',
    'model_reload': False,
    'model_quant': None,
    'system_prompt': prompts.SYSTEM_PROMPTS['BASE_EARN'],
    'multi-gpu':True,
    'dataset': 'data_quarterly_pit_indu_blended_base',
    'data_location': 'data_quarterly_pit_indu_refresh_blended.json'
}

In [6]:
run_name = f"{run_config['model_s3_loc']}_{run_config['dataset']}"
ir = model_inference.InferenceRun(run_name, run_config)

# Create the prompts and save to the Data folder
prompt_set = ir.create_all_prompts(force_refresh=True, is_save_prompts=True)

Requesting all datasets...
Saving data...


In [7]:
# Run the multi-gpu model with notebook_launcher
notebook_launcher(ir.run_multi_gpu, num_processes=torch.cuda.device_count())

Launching training on 4 GPUs.
deepseek7B
deepseek7B
deepseek7B
deepseek7B


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

Process ForkProcess-1 crashed: Process 793 got signal: 2


[2025-03-30 17:34:15,108] torch.distributed.elastic.multiprocessing.api: [WARNING] Closing process 796 via signal SIGINT


Process ForkProcess-3 crashed: Process 795 got signal: 2
Process ForkProcess-4 crashed: Process 796 got signal: 2
Process ForkProcess-2 crashed: Process 794 got signal: 2


[2025-03-30 17:34:45,140] torch.distributed.elastic.multiprocessing.api: [WARNING] Unable to shutdown process 793 via Signals.SIGINT, forcefully exiting via Signals.SIGKILL
[2025-03-30 17:34:45,368] torch.distributed.elastic.multiprocessing.api: [WARNING] Unable to shutdown process 794 via Signals.SIGINT, forcefully exiting via Signals.SIGKILL
[2025-03-30 17:34:45,608] torch.distributed.elastic.multiprocessing.api: [WARNING] Unable to shutdown process 795 via Signals.SIGINT, forcefully exiting via Signals.SIGKILL
[2025-03-30 17:34:45,848] torch.distributed.elastic.multiprocessing.api: [WARNING] Unable to shutdown process 796 via Signals.SIGINT, forcefully exiting via Signals.SIGKILL


SignalException: Process 717 got signal: 2

In [17]:

run_config = {
    'model_hf_id': 'deepseek-ai/DeepSeek-R1-Distill-Qwen-14B',
    'model_s3_loc': 'deepseek14Q',
    'model_reload': False,
    'model_quant': None,
    'system_prompt': prompts.SYSTEM_PROMPTS['CoT'],
    'multi-gpu':True,
    'dataset': 'data_annual_pit_spx',
    'data_location': 'data_annual_pit_spx.json'
}

run_config = {
    'model_hf_id': 'Qwen/Qwen2.5-7B-Instruct',
    'model_s3_loc': 'qwen',
    'model_reload': False,
    'model_quant': None,
    'system_prompt': prompts.SYSTEM_PROMPTS['BASE'],
    'multi-gpu':True,
    'dataset': 'data_quarterly_pit_indu',
    'data_location': 'data_quarterly_pit_indu.json'
}

# RUN 1
run_config = {
    'model_hf_id': 'meta-llama/Llama-3.2-3B-Instruct',
    'model_s3_loc': 'llama',
    'model_reload': False,
    'model_quant': None,
    'system_prompt': prompts.SYSTEM_PROMPTS['BASE'],
    'multi-gpu':True,
    'dataset': 'data_quarterly_pit_indu',
    'data_location': 'data_quarterly_pit_indu.json'
}

# RUN 2
run_config = {
    'model_hf_id': 'meta-llama/Llama-3.2-3B-Instruct',
    'model_s3_loc': 'llama',
    'model_reload': False,
    'model_quant': None,
    'system_prompt': prompts.SYSTEM_PROMPTS['CoT'],
    'multi-gpu':True,
    'dataset': 'data_quarterly_pit_indu',
    'data_location': 'data_quarterly_pit_indu.json'
}

# RUN 3
run_config = {
    'model_hf_id': 'deepseek-ai/DeepSeek-R1-Distill-Qwen-14B',
    'model_s3_loc': 'deepseek14Q',
    'model_reload': False,
    'model_quant': None,
    'system_prompt': prompts.SYSTEM_PROMPTS['BASE'],
    'multi-gpu':True,
    'dataset': 'data_quarterly_pit_indu',
    'data_location': 'data_quarterly_pit_indu.json'
}

# RUN 4
run_config = {
    'model_hf_id': 'deepseek-ai/DeepSeek-R1-Distill-Qwen-14B',
    'model_s3_loc': 'deepseek14Q',
    'model_reload': False,
    'model_quant': None,
    'system_prompt': prompts.SYSTEM_PROMPTS['CoT'],
    'multi-gpu':True,
    'dataset': 'data_quarterly_pit_indu',
    'data_location': 'data_quarterly_pit_indu.json'
}

# RUN 5 - failed
run_config = {
    'model_hf_id': 'deepseek-ai/DeepSeek-R1-Distill-Qwen-32B',
    'model_s3_loc': 'deepseek32',
    'model_reload': False,
    'model_quant': None,
    'system_prompt': prompts.SYSTEM_PROMPTS['CoT'],
    'multi-gpu':True,
    'dataset': 'data_quarterly_pit_indu',
    'data_location': 'data_quarterly_pit_indu.json'
}

# Run 6 - failed
run_config = {
    'model_hf_id': 'Qwen/Qwen2.5-7B-Instruct',
    'model_s3_loc': 'qwen',
    'model_reload': False,
    'model_quant': None,
    'system_prompt': prompts.SYSTEM_PROMPTS['CoTDetailed'],
    'multi-gpu':True,
    'dataset': 'data_quarterly_pit_indu',
    'data_location': 'data_quarterly_pit_indu.json'
}

run_config = {
    'model_hf_id': 'meta-llama/Llama-3.2-3B-Instruct',
    'model_s3_loc': 'llama',
    'model_reload': False,
    'model_quant': None,
    'system_prompt': prompts.SYSTEM_PROMPTS['BASE_EARN'],
    'multi-gpu':True,
    'dataset': 'data_quarterly_pit_indu_refresh_v2',
    'data_location': 'data_quarterly_pit_indu_refresh_v2.json'
}

# run_config = {
#     'model_hf_id': 'meta-llama/Llama-3.2-3B-Instruct',
#     'model_s3_loc': 'llama',
#     'model_reload': False,
#     'model_quant': None,
#     'system_prompt': prompts.SYSTEM_PROMPTS['COT_EARN'],
#     'multi-gpu':True,
#     'dataset': 'data_quarterly_pit_indu',
#     'data_location': 'data_quarterly_pit_indu.json'
# }



run_name = f"{run_config['model_s3_loc']}_{run_config['dataset']}"

In [18]:
ir = model_inference.InferenceRun(run_name, run_config)

In [19]:
# Create the prompts and save to the Data folder
prompts = ir.create_all_prompts(True)

Requesting all datasets...
Saving data...


In [20]:
# Run the multi-gpu model with notebook_launcher
notebook_launcher(ir.run_multi_gpu, num_processes=torch.cuda.device_count())

Launching training on 4 GPUs.
llama
llama
llama
llama


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

100%|█████████▉| 888/891 [27:10<00:04,  1.60s/it]

Finished run...


892it [27:19,  1.82s/it]                         

Finished run...
Finished run...
Gathered results...Gathered results...Gathered results...Gathered results...



Finished run in 0:27:51.318931
Called Save run
called log
Saved bclarke16/tmp/fs/logs/Results_2025-03-30 09:15:48.238865.json
Run Completed!


892it [27:51,  1.87s/it]
[2025-03-30 09:44:39,571] torch.distributed.elastic.multiprocessing.api: [WARNING] Closing process 257 via signal SIGTERM
[2025-03-30 09:44:39,572] torch.distributed.elastic.multiprocessing.api: [WARNING] Closing process 258 via signal SIGTERM
[2025-03-30 09:44:39,573] torch.distributed.elastic.multiprocessing.api: [WARNING] Closing process 260 via signal SIGTERM


In [11]:
ir = model_inference.InferenceRun(run_name, run_config)

In [12]:
p1 = ir.create_all_prompts(True)

Requesting all datasets...
Saving data...


In [11]:
len(p1)

896

In [12]:
model = ir.load_model_from_storage(ir.model_s3_loc)

deepseek32


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Qwen2Config {
  "_attn_implementation_autoset": true,
  "_name_or_path": "deepseek32",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 5120,
  "initializer_range": 0.02,
  "intermediate_size": 27648,
  "max_position_embeddings": 131072,
  "max_window_layers": 64,
  "model_type": "qwen2",
  "num_attention_heads": 40,
  "num_hidden_layers": 64,
  "num_key_value_heads": 8,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  },
  "rms_norm_eps": 1e-05,

In [23]:
from transformers import AutoTokenizer

In [24]:
tokenizer = AutoTokenizer.from_pretrained(ir.model_hf_id)

In [28]:
output_result = ir.run_model(p1[0]['prompt'],tokenizer,model)

running model...


In [8]:
output_result = {"date": "2020-02-06", "security": "MMM UN Equity", "response": "Here is the JSON response:\n\n```json\n{\n  \"decision\": \"BUY\",\n  \"confidence score\": 80,\n  \"reason\": \"Gross profit and EPS have increased over time, indicating a strong financial performance\"\n}\n```\n\nI have computed the following financial ratios:\n\n1. Gross Margin: \n   - 2020: 3.786000e+09 / 8.111000e+09 = 0.466\n   - 2019: 3.803000e+09 / 7.991000e+09 = 0.477\n   - 2018: 3.858000e+09 / 8.171000e+09 = 0.471\n   - 2017: 3.553000e+09 / 7.863000e+09 = 0.453\n   - 2016: 3.885000e+09 / 7.945000e+09 = 0.492\n\nThe gross margin has been increasing over time, indicating a strong financial performance.\n\n2. EPS:\n   - 2020: 9.690000e+08 / 1.012600e+10 = 0.953\n   - 2019: 1.583000e+09 / 1.076400e+10 = 1.465\n   - 2018: 1.127000e+09 / 1.014200e+10 = 1.117\n   - 2017: 1.347000e+09 / 9.848000e+09 = 0.137\n   - 2016: 1.543000e+09 / 9.848000e+09 = 0.156\n\nThe EPS has been increasing over time, indicating a strong financial performance.\n\n3. Current Ratio:\n   - 2020: 2.441000e+09 / 9.222000e+09 = 0.264\n   - 2019: 1.588000e+09 / 7.821000e+09 = 0.201\n   - 2018: 1.131000e+09 / 7.265000e+09 = 0.157\n   - 2017: 8.930000e+08 / 7.244000e+09 = 0.123\n   - 2016: 8.910000e+08 / 5.020000e+09 = 0.177\n\nThe current ratio has been increasing over time, indicating a strong financial performance.\n\nBased on these financial ratios, the company has been performing well financially and has a strong track record of increasing gross profit and EPS over time. Therefore, I recommend a BUY decision with a confidence score of 80."}

In [10]:
output_result['response']

'Here is the JSON response:\n\n```json\n{\n  "decision": "BUY",\n  "confidence score": 80,\n  "reason": "Gross profit and EPS have increased over time, indicating a strong financial performance"\n}\n```\n\nI have computed the following financial ratios:\n\n1. Gross Margin: \n   - 2020: 3.786000e+09 / 8.111000e+09 = 0.466\n   - 2019: 3.803000e+09 / 7.991000e+09 = 0.477\n   - 2018: 3.858000e+09 / 8.171000e+09 = 0.471\n   - 2017: 3.553000e+09 / 7.863000e+09 = 0.453\n   - 2016: 3.885000e+09 / 7.945000e+09 = 0.492\n\nThe gross margin has been increasing over time, indicating a strong financial performance.\n\n2. EPS:\n   - 2020: 9.690000e+08 / 1.012600e+10 = 0.953\n   - 2019: 1.583000e+09 / 1.076400e+10 = 1.465\n   - 2018: 1.127000e+09 / 1.014200e+10 = 1.117\n   - 2017: 1.347000e+09 / 9.848000e+09 = 0.137\n   - 2016: 1.543000e+09 / 9.848000e+09 = 0.156\n\nThe EPS has been increasing over time, indicating a strong financial performance.\n\n3. Current Ratio:\n   - 2020: 2.441000e+09 / 9.22200

In [11]:
import json

In [20]:
def format_json(llm_output):
    # remove all the broken lines
    form = llm_output.replace('\n','')
    # Find the start and end of the JSON input
    #try:
    soj = form.find('```json')
    eoj = form.find('}```')

    if eoj == -1:
        eoj = len(llm_output)
        llm_output = llm_output + '}```'
    # Pull out the additional context
    additional = form[:soj]
    additional += form[eoj + 4:]
    json_obj = json.loads(form[soj + 7:eoj + 1])
    json_obj['AdditionalContext'] = additional
    return json_obj
    #except:
    #    return llm_output

In [21]:
format_json(output_result['response'])

{  "decision": "BUY",  "confidence score": 80,  "reason": "Gross profit and EPS have increased over time, indicating a strong financial performance"}


{'decision': 'BUY',
 'confidence score': 80,
 'reason': 'Gross profit and EPS have increased over time, indicating a strong financial performance',
 'AdditionalContext': 'Here is the JSON response:I have computed the following financial ratios:1. Gross Margin:    - 2020: 3.786000e+09 / 8.111000e+09 = 0.466   - 2019: 3.803000e+09 / 7.991000e+09 = 0.477   - 2018: 3.858000e+09 / 8.171000e+09 = 0.471   - 2017: 3.553000e+09 / 7.863000e+09 = 0.453   - 2016: 3.885000e+09 / 7.945000e+09 = 0.492The gross margin has been increasing over time, indicating a strong financial performance.2. EPS:   - 2020: 9.690000e+08 / 1.012600e+10 = 0.953   - 2019: 1.583000e+09 / 1.076400e+10 = 1.465   - 2018: 1.127000e+09 / 1.014200e+10 = 1.117   - 2017: 1.347000e+09 / 9.848000e+09 = 0.137   - 2016: 1.543000e+09 / 9.848000e+09 = 0.156The EPS has been increasing over time, indicating a strong financial performance.3. Current Ratio:   - 2020: 2.441000e+09 / 9.222000e+09 = 0.264   - 2019: 1.588000e+09 / 7.821000e+09

In [14]:
ir.save_run({'test':'1234'})

Saved s3://awmgd-prod-finml-sandbox-user/bclarke16/tmp/fs/logs/results - llama - data_quarterly_pit_indu
Run Completed!


In [9]:
helper = mh.ModelHelper('tmp/fs')

In [10]:
helper.clear_folder('llama')